# GHZ Scaling & Variational Optimization with CUDA-Q

This notebook demonstrates devqubit tracking for **entanglement scaling** and **variational optimization** using NVIDIA CUDA-Q.

**What you'll see:**
- Kernel-based circuit definition with `@cudaq.kernel`
- GHZ population scaling as qubit count grows under noise
- Multi-target validation with `set_target()` tracking
- Variational optimization via `cudaq.observe()`
- Baseline verification and run comparison

**Two projects:**
- `ghz_scaling` — entanglement population under noise, multi-target validation
- `vqe_ising` — variational ground-state search with `cudaq.observe()`

**CUDA-Q specifics:** Unlike Qiskit/Cirq/PennyLane where circuits are data structures,
CUDA-Q kernels are **compiled functions** decorated with `@cudaq.kernel`. The SDK handles
compilation to quantum IR, target dispatch, and GPU acceleration. devqubit's `run.wrap(cudaq)`
intercepts `sample()`, `observe()`, and `set_target()` transparently.

In [ ]:
from importlib.metadata import entry_points

if not any(ep.name == "cudaq" for ep in entry_points(group="devqubit.adapters")):
    raise ImportError(
        "devqubit CUDA-Q adapter is not installed.\n"
        "Install with: pip install 'devqubit[cudaq]'"
    )
else:
    print("CUDA-Q adapter available!")

In [ ]:
from pathlib import Path
import shutil
import numpy as np

import cudaq

from devqubit import Config, set_config, track
from devqubit.compare import diff, verify_baseline, VerifyPolicy
from devqubit.runs import (
    set_baseline,
    load_run,
    list_runs,
)

### Setup

In [ ]:
PROJECT_GHZ = "ghz_scaling"
PROJECT_VQE = "vqe_ising"
SHOTS = 4096
SEED = 42

WORKSPACE = Path(".devqubit_cudaq_demo")
if WORKSPACE.exists():
    shutil.rmtree(WORKSPACE)

set_config(Config(root_dir=WORKSPACE))

cudaq.set_random_seed(SEED)
np.random.seed(SEED)
print(f"Workspace: {WORKSPACE.resolve()}")

### 🌐 Launch Web UI (Optional)

Start the devqubit web UI **now** to watch experiments appear in real time as you run the cells below.

> Run the cell below, then open the URL in a browser and keep it open while you execute the rest of the notebook.
> New runs will appear automatically after each experiment completes.

In [ ]:
from devqubit.ui import run_server

run_server(workspace=str(WORKSPACE))

---
## Part A: GHZ Population Scaling

### 1. GHZ Kernels and Utilities

CUDA-Q kernels are compiled at definition time, so we create one per qubit count
rather than passing $n$ as a runtime parameter.

**GHZ population:** For an ideal $n$-qubit GHZ state the only outcomes are
$|0\rangle^{\otimes n}$ and $|1\rangle^{\otimes n}$, each with probability 0.5.
We measure $P_{\text{pop}} = P(\text{all-zero}) + P(\text{all-one})$ — the Z-basis
population, not full state fidelity (which would require coherence measurements).

In [ ]:
def make_ghz_kernel(n: int):
    """Return an n-qubit GHZ kernel with measurements."""
    if n < 2:
        raise ValueError("n must be >= 2")

    @cudaq.kernel
    def ghz():
        q = cudaq.qvector(n)
        h(q[0])
        for i in range(n - 1):
            x.ctrl(q[i], q[i + 1])
        mz(q)

    return ghz

def ghz_population(counts: dict, n_qubits: int, total_shots: int) -> float:
    """GHZ Z-basis population: P(all-0) + P(all-1). Not full state fidelity."""
    p0 = counts.get("0" * n_qubits, 0) / total_shots
    p1 = counts.get("1" * n_qubits, 0) / total_shots
    return p0 + p1


def ghz_balance(counts: dict, n_qubits: int) -> float:
    """Asymmetry |P(all-0) - P(all-1)| / (P(all-0) + P(all-1)). 0 = perfect."""
    p0 = counts.get("0" * n_qubits, 0)
    p1 = counts.get("1" * n_qubits, 0)
    total = p0 + p1
    return abs(p0 - p1) / total if total > 0 else 1.0


def extract_counts(result) -> dict:
    """Extract bitstring counts from cudaq.SampleResult."""
    return {bs: count for bs, count in result.items()}


# Quick sanity check
kernel_4 = make_ghz_kernel(4)
test_result = cudaq.sample(kernel_4, shots_count=1000)
test_counts = extract_counts(test_result)
print(f"GHZ-4 test: {test_counts}")
print(f"Population: {ghz_population(test_counts, 4, 1000):.4f}")

### 2. Ideal Baseline

Establish the noiseless reference on `qpp-cpu` (state-vector simulator).
`run.wrap(cudaq)` intercepts `sample()` and captures kernel structure,
measurement results, and target configuration automatically.

In [ ]:
cudaq.set_target("qpp-cpu")

with track(project=PROJECT_GHZ, run_name="ideal_baseline") as run:
    executor = run.wrap(cudaq)

    run.log_params({
        "n_qubits": 4, "shots": SHOTS,
        "noise_type": "ideal", "error_rate": 0.0,
        "target": "qpp-cpu",
    })
    run.set_tags({"role": "baseline", "noise": "ideal"})

    kernel = make_ghz_kernel(4)
    result = executor.sample(kernel, shots_count=SHOTS)
    counts = extract_counts(result)

    population = ghz_population(counts, 4, SHOTS)
    balance = ghz_balance(counts, 4)
    run.log_metrics({"ghz_population": population, "balance": balance, "unique_outcomes": len(counts)})

    ghz_baseline_id = run.run_id

print(f"Baseline run:   {ghz_baseline_id[:16]}")
print(f"GHZ-4 population: {population:.4f}")
print(f"Counts:         {counts}")

### 3. Qubit Count Scaling

How does GHZ population degrade as we increase qubit count under depolarizing noise?
GHZ depth is linear in $n$, and population decays roughly as $F \approx (1-p)^d$.

Noise simulation requires `density-matrix-cpu` — the state-vector simulator
(`qpp-cpu`) silently ignores noise models. For two-qubit gates we use
`Depolarization2`.

In [ ]:
cudaq.set_target("density-matrix-cpu")

qubit_counts = [2, 4, 6, 8]
scale_results = []

noise = cudaq.NoiseModel()
noise.add_all_qubit_channel("h", cudaq.DepolarizationChannel(0.005))
noise.add_all_qubit_channel("x", cudaq.Depolarization2(0.01), 1)

print("Qubit Count Scaling (depol 1q=0.005, 2q=0.01)")
print("=" * 55)

for nq in qubit_counts:
    with track(
        project=PROJECT_GHZ,
        run_name=f"ghz_{nq}q_depol",
        group_id="qubit_scaling",
        group_name="Qubit Count Scaling",
    ) as run:
        executor = run.wrap(cudaq)

        run.log_params({
            "n_qubits": nq,
            "shots": SHOTS,
            "noise_type": "depolarizing",
            "error_rate_1q": 0.005,
            "error_rate_2q": 0.01,
            "target": "density-matrix-cpu",
        })
        run.set_tags({"experiment": "scaling", "noise": "depolarizing"})

        kernel = make_ghz_kernel(nq)
        result = executor.sample(kernel, shots_count=SHOTS, noise_model=noise)

        counts = extract_counts(result)
        pop = ghz_population(counts, nq, SHOTS)
        bal = ghz_balance(counts, nq)

        run.log_metrics({
            "ghz_population": pop, "balance": bal,
            "unique_outcomes": len(counts), "circuit_depth": nq,
        })

        scale_results.append(
            {
                "n_qubits": nq,
                "ghz_population": pop,
                "balance": bal,
                "run_id": run.run_id
            }
        )
        print(f"  n={nq:2d}  pop={pop:.4f}  balance={bal:.4f}  outcomes={len(counts)}")

In [ ]:
print("Population Scaling Summary")
print("-" * 40)

for r in scale_results:
    bar = "\u2588" * int(r["ghz_population"] * 40)
    print(f"  n={r['n_qubits']:2d}  {r['ghz_population']:.4f}  {bar}")

if len(scale_results) >= 2:
    f_first = scale_results[0]["ghz_population"]
    f_last = scale_results[-1]["ghz_population"]
    n_first = scale_results[0]["n_qubits"]
    n_last = scale_results[-1]["n_qubits"]
    if f_last > 0 and f_first > 0:
        decay = -(np.log(f_last) - np.log(f_first)) / (n_last - n_first)
        print(f"\nEstimated decay rate: {decay:.4f} per qubit")

### 4. Multi-Target Validation

CUDA-Q supports multiple simulation backends via `set_target()`. devqubit
tracks these calls as runtime configuration events and automatically
invalidates the cached device snapshot, so each run's metadata accurately
reflects which target produced the results.

We compare `qpp-cpu` (state vector) with `density-matrix-cpu` (density matrix)
on the same noiseless GHZ-4 kernel — they should agree within shot noise.

In [ ]:
target_results = []

for target_name in ["qpp-cpu", "density-matrix-cpu"]:
    with track(
        project=PROJECT_GHZ,
        run_name=f"ideal_{target_name}",
        group_id="target_validation",
        group_name="Multi-Target Validation",
    ) as run:
        executor = run.wrap(cudaq)
        executor.set_target(target_name)

        run.log_params({
            "n_qubits": 4, "shots": SHOTS,
            "noise_type": "ideal", "error_rate": 0.0,
            "target": target_name,
        })
        run.set_tags({"experiment": "target_validation", "target": target_name})

        kernel = make_ghz_kernel(4)
        result = executor.sample(kernel, shots_count=SHOTS)
        counts = extract_counts(result)

        pop = ghz_population(counts, 4, SHOTS)
        run.log_metrics({"ghz_population": pop, "unique_outcomes": len(counts)})

        target_results.append(
            {"target": target_name, "ghz_population": pop, "counts": counts, "run_id": run.run_id}
        )
        print(f"  {target_name:25s}  pop={pop:.4f}  counts={counts}")

### 5. GHZ: Baseline, Verify, and Compare

Set the ideal run as baseline and verify the density-matrix target run
against it — same kernel, results should agree within shot noise.

In [ ]:
set_baseline(PROJECT_GHZ, ghz_baseline_id)
print(f"Baseline set: {ghz_baseline_id[:16]}")

dm_run_id = target_results[1]["run_id"]

policy = VerifyPolicy(
    params_must_match=False,
    program_must_match=True,
    tvd_max=0.10,
)

result = verify_baseline(
    dm_run_id,
    project=PROJECT_GHZ,
    policy=policy,
)

print(result)

In [ ]:
worst_noise = sorted(scale_results, key=lambda x: x["ghz_population"])[0]
comparison = diff(ghz_baseline_id, worst_noise["run_id"])
print(comparison)

---
## Part B: Variational Optimization

### 6. VQE with `cudaq.observe()`

CUDA-Q's `observe()` computes Hamiltonian expectation values directly —
no explicit measurement needed in the kernel. This is the foundation for
VQE-style algorithms and a key differentiator of the CUDA-Q API.

We optimize a 2-qubit variational circuit to find the ground state
of $H = -Z_0 Z_1 - 0.5\,(X_0 + X_1)$, a transverse-field Ising model.
The exact ground-state energy is $-\sqrt{2} \approx -1.414$.

In [ ]:
cudaq.set_target("qpp-cpu")

@cudaq.kernel
def ansatz(theta: float, phi: float):
    q = cudaq.qvector(2)
    ry(theta, q[0])
    ry(phi, q[1])
    x.ctrl(q[0], q[1])


hamiltonian = (
    -1.0 * cudaq.spin.z(0) * cudaq.spin.z(1)
    - 0.5 * cudaq.spin.x(0)
    - 0.5 * cudaq.spin.x(1)
)

exact_gs = -np.sqrt(2.0)
print(f"Hamiltonian: -Z0*Z1 - 0.5*(X0 + X1)")
print(f"Exact ground-state energy: {exact_gs:.6f}")

In [ ]:
N_STEPS = 50
LR = 0.1

with track(project=PROJECT_VQE, run_name="baseline") as run:
    executor = run.wrap(cudaq)

    run.log_params({
        "n_qubits": 2,
        "method": "observe",
        "optimizer": "finite_diff",
        "lr": LR,
        "n_steps": N_STEPS,
        "hamiltonian": "-ZZ - 0.5*(X+X)",
        "target": "qpp-cpu",
    })
    run.set_tags({"role": "baseline", "method": "observe"})

    rng = np.random.default_rng(SEED)
    params = rng.uniform(-np.pi, np.pi, size=2)
    best_energy = float("inf")

    for step in range(N_STEPS):
        grad = np.zeros(2)
        eps = 0.1
        for j in range(2):
            params_plus = params.copy()
            params_plus[j] += eps
            params_minus = params.copy()
            params_minus[j] -= eps
            e_plus = executor.observe(ansatz, hamiltonian, *params_plus).expectation()
            e_minus = executor.observe(ansatz, hamiltonian, *params_minus).expectation()
            grad[j] = (e_plus - e_minus) / (2 * eps)

        params -= LR * grad
        energy = executor.observe(ansatz, hamiltonian, *params).expectation()
        best_energy = min(best_energy, energy)

        run.log_metric("energy", energy, step=step)
        run.log_metric("best_energy", best_energy, step=step)

    error = abs(best_energy - exact_gs)
    run.log_metrics({
        "final_energy": best_energy,
        "within_tolerance": error < 0.05,
        "energy_error": error,
    })

    vqe_baseline_id = run.run_id

print(f"Final energy:  {best_energy:.6f}")
print(f"Exact energy:  {exact_gs:.6f}")
print(f"Error:         {error:.6f}")

### 7. VQE: Baseline and Candidate Verification

Run the same optimization with a different random seed and verify
the candidate converges to a comparable energy.

In [ ]:
set_baseline(PROJECT_VQE, vqe_baseline_id)
print(f"VQE baseline set: {vqe_baseline_id[:16]}")

with track(project=PROJECT_VQE, run_name="candidate") as run:
    executor = run.wrap(cudaq)

    run.log_params({
        "n_qubits": 2,
        "method": "observe",
        "optimizer": "finite_diff",
        "lr": LR,
        "n_steps": N_STEPS,
        "hamiltonian": "-ZZ - 0.5*(X+X)",
        "target": "qpp-cpu",
    })
    run.set_tags({"role": "candidate", "method": "observe"})

    rng2 = np.random.default_rng(SEED + 1)
    params = rng2.uniform(-np.pi, np.pi, size=2)
    best_energy = float("inf")

    for step in range(N_STEPS):
        grad = np.zeros(2)
        eps = 0.1
        for j in range(2):
            params_plus = params.copy()
            params_plus[j] += eps
            params_minus = params.copy()
            params_minus[j] -= eps
            e_plus = executor.observe(ansatz, hamiltonian, *params_plus).expectation()
            e_minus = executor.observe(ansatz, hamiltonian, *params_minus).expectation()
            grad[j] = (e_plus - e_minus) / (2 * eps)

        params -= LR * grad
        energy = executor.observe(ansatz, hamiltonian, *params).expectation()
        best_energy = min(best_energy, energy)

        run.log_metric("energy", energy, step=step)
        run.log_metric("best_energy", best_energy, step=step)

    error = abs(best_energy - exact_gs)
    run.log_metrics({
        "final_energy": best_energy,
        "within_tolerance": error < 0.05,
        "energy_error": error,
    })

    vqe_candidate_id = run.run_id

print(f"Candidate energy: {best_energy:.6f}  (error: {error:.6f})")

In [ ]:
policy = VerifyPolicy(
    params_must_match=True,
    program_must_match=True,
    tvd_max=0.15,
)

result = verify_baseline(
    vqe_candidate_id,
    project=PROJECT_VQE,
    policy=policy,
)

print(result)

In [ ]:
comparison = diff(vqe_baseline_id, vqe_candidate_id)
print(comparison)

### 8. All Projects Overview

In [ ]:
for project in [PROJECT_GHZ, PROJECT_VQE]:
    print(f"\nProject: {project}")
    print("-" * 60)

    for info in list_runs(project=project):
        rec = load_run(info["run_id"])
        role = rec.tags.get("role", "-")

        if project == PROJECT_GHZ:
            val = rec.metrics.get("ghz_population", 0)
            label = f"fid={val:.4f}"
        else:
            val = rec.metrics.get("final_energy", rec.metrics.get("energy", 0))
            label = f"E={val:.4f}"

        print(f"  {rec.run_id[:12]}  {role:10s}  {rec.run_name:25s}  {label}")

### Summary

| Feature | What we used it for |
|---------|--------------------|
| `track()` | Create tracked runs with automatic finalization |
| `run.wrap(cudaq)` | Capture `sample()` and `observe()` calls |
| `executor.set_target()` | Track backend switches with snapshot invalidation |
| `executor.observe()` | Track Hamiltonian expectation values (VQE) |
| `group_id` | Organize qubit scaling sweep |
| `set_baseline()` / `verify_baseline()` | Quality checks per project |
| `diff()` | Human-readable run comparison |

**CUDA-Q specifics demonstrated:**
- `@cudaq.kernel` — circuits as compiled functions
- `set_target()` tracking — devqubit captures target switches as runtime events
- `cudaq.observe()` — analytic expectation values for variational algorithms
- `cudaq.NoiseModel` — noise channels passed to `sample()`

In [ ]:
shutil.rmtree(WORKSPACE)
print("Done!")